In [1]:
from dotenv import load_dotenv

load_dotenv()

True

In [2]:
import requests 

docs_url = 'https://github.com/alexeygrigorev/llm-rag-workshop/raw/main/notebooks/documents.json'
docs_response = requests.get(docs_url)
documents_raw = docs_response.json()

documents = []

for course in documents_raw:
    course_name = course['course']

    for doc in course['documents']:
        doc['course'] = course_name
        documents.append(doc)

In [3]:
from minsearch import AppendableIndex

index = AppendableIndex(   #same as index , except we can add documents into the index
    text_fields=["question", "text", "section"],
    keyword_fields=["course"]
)

index.fit(documents)

In [4]:
def search(query):
    boost = {'question': 3.0, 'section': 0.5}

    results = index.search(
        query=query,
        filter_dict = {'course': 'data-engineering-zoomcamp'},
        boost_dict=boost,
        num_results=5,
        output_ids=True        
            )

    return results

In [5]:
question = 'can i still join the course?'

In [6]:
def build_prompt(query, search_results):
    context = "" 
    
    for doc in search_results:
        context = context + f"section: {doc['section']}\nquestion: {doc['question']}\nanswer: {doc['text']}\n\n"

    print(context)
    prompt = f""" 
    You're a course teaching assistant. Answer the QUESTION based on the CONTEXT from the FAQ database. 
    Use only the facts from the CONTEXT when answering the question. If the CONTEXT doesn't contain the answer, output NONE

    <QUESTION>{query}</QUESTION>
    <CONTEXT>{context}</CONTEXT>
    """.strip()

    return prompt



In [7]:
from openai import OpenAI
client = OpenAI()

def llm(prompt):
    response = client.chat.completions.create(
        model='gpt-4o-mini',
        messages=[{"role": "user", "content": prompt}]
    )

    return response.choices[0].message.content

In [8]:
def rag(query):
    search_results = search(query)
    prompt = build_prompt(query, search_results)
    answer = llm(prompt)

    return answer

In [10]:
rag(question)

section: General course-related questions
question: Course - Can I still join the course after the start date?
answer: Yes, even if you don't register, you're still eligible to submit the homeworks.
Be aware, however, that there will be deadlines for turning in the final projects. So don't leave everything for the last minute.

section: General course-related questions
question: Certificate - Can I follow the course in a self-paced mode and get a certificate?
answer: No, you can only get a certificate if you finish the course with a “live” cohort. We don't award certificates for the self-paced mode. The reason is you need to peer-review capstone(s) after submitting a project. You can only peer-review projects at the time the course is running.

section: General course-related questions
question: Course - Can I follow the course after it finishes?
answer: Yes, we will keep all the materials after the course finishes, so you can follow the course at your own pace after it finishes.
You c

"Yes, you can still join the course after the start date. Even if you don't register, you're still eligible to submit the homeworks. However, be aware that there will be deadlines for turning in the final projects."

### Agentic RAG

In [12]:
prompt_template = """
You're a course teaching assistant.

You're given a QUESTION from a course student and that you need to answer with your own knowledge and provided CONTEXT.
At the beginning the context is EMPTY.

<QUESTION>
{question}
</QUESTION>

<CONTEXT> 
{context}
</CONTEXT>

If CONTEXT is EMPTY, you can use our FAQ database.
In this case, use the following output template:

{{
"action": "SEARCH",
"reasoning": "<add your reasoning here>"
}}

If you can answer the QUESTION using CONTEXT, use this template:

{{
"action": "ANSWER",
"answer": "<your answer>",
"source": "CONTEXT"
}}

If the context doesn't contain the answer, use your own knowledge to answer the question

{{
"action": "ANSWER",
"answer": "<your answer>",
"source": "OWN_KNOWLEDGE"
}}
""".strip()

In [13]:
question = 'Can I still join the course?'
context = 'EMPTY'

In [14]:
prompt = prompt_template.format(question=question, context=context)

In [15]:
answer_json = llm(prompt)

In [16]:
import json

answer = json.loads(answer_json)

In [17]:
answer

{'action': 'SEARCH',
 'reasoning': 'The context is empty, so I need to refer to the FAQ database to check if there are any guidelines regarding joining the course at this point.'}

In [18]:

def build_context(search_results):
    context = ""

    for doc in search_results:
        context = context + f"section: {doc['section']}\nquestion: {doc['question']}\nanswer: {doc['text']}\n\n"
    
    return context.strip()

In [19]:
search_results = search(question)
context = build_context(search_results)
prompt = prompt_template.format(question=question, context=context)

In [20]:
answer_json = llm(prompt)
answer_json

'{\n"action": "ANSWER",\n"answer": "Yes, you can still join the course even after the start date. Although registration is encouraged, you are eligible to submit homework assignments regardless of whether you registered. However, be mindful of any deadlines for final projects, as it\'s best not to procrastinate.",\n"source": "CONTEXT"\n}'

In [22]:
print(answer_json)

{
"action": "ANSWER",
"answer": "Yes, you can still join the course even after the start date. Although registration is encouraged, you are eligible to submit homework assignments regardless of whether you registered. However, be mindful of any deadlines for final projects, as it's best not to procrastinate.",
"source": "CONTEXT"
}


## Agentic Search

In [23]:
def dedup(seq):
    seen = set()
    result = []
    for el in seq:
        _id = el['_id']
        if _id in seen:
            continue
        seen.add(_id)
        result.append(el)
    return result

In [24]:
prompt_template = """
You're a course teaching assistant.

You're given a QUESTION from a course student and that you need to answer with your own knowledge and provided CONTEXT.

The CONTEXT is build with the documents from our FAQ database.
SEARCH_QUERIES contains the queries that were used to retrieve the documents
from FAQ to and add them to the context.
PREVIOUS_ACTIONS contains the actions you already performed.

At the beginning the CONTEXT is empty.

You can perform the following actions:

- Search in the FAQ database to get more data for the CONTEXT
- Answer the question using the CONTEXT
- Answer the question using your own knowledge

For the SEARCH action, build search requests based on the CONTEXT and the QUESTION.
Carefully analyze the CONTEXT and generate the requests to deeply explore the topic. 

Don't use search queries used at the previous iterations.

Don't repeat previously performed actions.

Don't perform more than {max_iterations} iterations for a given student question.
The current iteration number: {iteration_number}. If we exceed the allowed number 
of iterations, give the best possible answer with the provided information.

Output templates:

If you want to perform search, use this template:

{{
"action": "SEARCH",
"reasoning": "<add your reasoning here>",
"keywords": ["search query 1", "search query 2", ...]
}}

If you can answer the QUESTION using CONTEXT, use this template:

{{
"action": "ANSWER_CONTEXT",
"answer": "<your answer>",
"source": "CONTEXT"
}}

If the context doesn't contain the answer, use your own knowledge to answer the question

{{
"action": "ANSWER",
"answer": "<your answer>",
"source": "OWN_KNOWLEDGE"
}}

<QUESTION>
{question}
</QUESTION>

<SEARCH_QUERIES>
{search_queries}
</SEARCH_QUERIES>

<CONTEXT> 
{context}
</CONTEXT>

<PREVIOUS_ACTIONS>
{previous_actions}
</PREVIOUS_ACTIONS>
""".strip()

In [25]:
question = 'how do I do well on module 1'
max_iterations = 3
iteration_number = 0
search_queries = []
search_results  = []
previous_actions = []

In [26]:
context = build_context(search_results)

prompt = prompt_template.format(
    question=question,
    context=context,
    search_queries="\n".join(search_queries),
    previous_actions='\n'.join([json.dumps(a) for a in previous_actions]),
    max_iterations=max_iterations,
    iteration_number=iteration_number
)


In [27]:
answer_json = llm(prompt)

In [29]:
answer = json.loads(answer_json)

In [28]:
print(answer_json)

{
"action": "SEARCH",
"reasoning": "The question is about performing well in a specific module, which could involve study tips, resources, or module structure. I need to gather relevant information from the FAQ database that may provide insights on how to succeed in Module 1.",
"keywords": ["how to succeed in module 1", "tips for excelling in first module", "study strategies for module 1"]
}


In [30]:

previous_actions.append(answer)

In [31]:

keywords = answer['keywords']

In [32]:

for kw in keywords:
    search_queries.append(kw)
    sr = search(kw)
    search_results.extend(sr)

In [33]:

search_results = dedup(search_results)

In [34]:
iteration_number = 2

context = build_context(search_results)

prompt = prompt_template.format(
    question=question,
    context=context,
    search_queries="\n".join(search_queries),
    previous_actions='\n'.join([json.dumps(a) for a in previous_actions]),
    max_iterations=max_iterations,
    iteration_number=iteration_number
)

In [35]:
answer_json = llm(prompt)

In [36]:
print(answer_json)

{
"action": "SEARCH",
"reasoning": "To further assist the student on how to do well on Module 1, I need to check for specific strategies or advice that might be mentioned in the FAQ database related to study methods, assessment tips, or resources for Module 1, particularly in Docker and Terraform which cover databases and Python integration.", 
"keywords": ["strategies for success in Docker and Terraform", "advice for studying Docker and Terraform", "resources for Module 1 Docker and Terraform"]
}


In [39]:
question = "what do I need to do to be successful at module 1?"

search_queries = []
search_results = []
previous_actions = []

iteration = 0

while True:
    print(f'ITERATION #{iteration}...')

    context = build_context(search_results)
    prompt = prompt_template.format(
        question=question,
        context=context,
        search_queries="\n".join(search_queries),
        previous_actions='\n'.join([json.dumps(a) for a in previous_actions]),
        max_iterations=3,
        iteration_number=iteration
    )

    print(prompt)

    answer_json = llm(prompt)
    answer = json.loads(answer_json)
    print(json.dumps(answer, indent=2))

    previous_actions.append(answer)

    action = answer['action']
    if action != 'SEARCH':
        break

    keywords = answer['keywords']
    search_queries = list(set(search_queries) | set(keywords))
    
    for k in keywords:
        res = search(k)
        search_results.extend(res)

    search_results = dedup(search_results)
    
    iteration = iteration + 1
    if iteration >= 4:
        break

    print()

ITERATION #0...
You're a course teaching assistant.

You're given a QUESTION from a course student and that you need to answer with your own knowledge and provided CONTEXT.

The CONTEXT is build with the documents from our FAQ database.
SEARCH_QUERIES contains the queries that were used to retrieve the documents
from FAQ to and add them to the context.
PREVIOUS_ACTIONS contains the actions you already performed.

At the beginning the CONTEXT is empty.

You can perform the following actions:

- Search in the FAQ database to get more data for the CONTEXT
- Answer the question using the CONTEXT
- Answer the question using your own knowledge

For the SEARCH action, build search requests based on the CONTEXT and the QUESTION.
Carefully analyze the CONTEXT and generate the requests to deeply explore the topic. 

Don't use search queries used at the previous iterations.

Don't repeat previously performed actions.

Don't perform more than 3 iterations for a given student question.
The current 

In [40]:
answer

{'action': 'ANSWER',
 'answer': "To be successful in Module 1, which focuses on Docker and Terraform, consider the following strategies:\n\n1. **Hands-on Practice**: Both Docker and Terraform are tools best learned through real-world practice. Set up your own projects to get familiar with the commands and workflows.\n\n2. **Understand Core Concepts**: Make sure you comprehend the core concepts, such as the architecture of Docker containers and Terraform's infrastructure as code philosophy. This foundational knowledge will help you when dealing with more complex tasks.\n\n3. **Utilize Documentation**: Both Docker and Terraform have extensive documentation. Use these resources to clarify doubts and learn about new features and best practices.\n\n4. **Engage with the Community**: Participate in forums, discussions, or study groups related to Docker and Terraform. This interaction can help you gain insights and resolve challenges you encounter.\n\n5. **Focus on Error Handling**: Familiariz

## Function calling ("tool use")

In [41]:

def search(query):
    boost = {'question': 3.0, 'section': 0.5}

    results = index.search(
        query=query,
        filter_dict={'course': 'data-engineering-zoomcamp'},
        boost_dict=boost,
        num_results=5,
        output_ids=True
    )

    return results

In [42]:
search_tool = {
    "type": "function",
    "name": "search",
    "description": "Search the FAQ database",
    "parameters": {
        "type": "object",
        "properties": {
            "query": {
                "type": "string",
                "description": "Search query text to look up in the course FAQ."
            }
        },
        "required": ["query"],
        "additionalProperties": False
    }
}

In [43]:

def do_call(tool_call_response):
    function_name = tool_call_response.name
    arguments = json.loads(tool_call_response.arguments)

    f = globals()[function_name]
    result = f(**arguments)

    return {
        "type": "function_call_output",
        "call_id": tool_call_response.call_id,
        "output": json.dumps(result, indent=2),
    }

In [44]:
question = "How do I do well in module 1?"

developer_prompt = """
You're a course teaching assistant. 
You're given a question from a course student and your task is to answer it.
If you look up something in FAQ, convert the student question into multiple queries.
""".strip()

tools = [search_tool]

chat_messages = [
    {"role": "developer", "content": developer_prompt},
    {"role": "user", "content": question}
]

response = client.responses.create(
    model='gpt-4o-mini',
    input=chat_messages,
    tools=tools
)
response.output

[ResponseFunctionToolCall(arguments='{"query":"How to do well in module 1?"}', call_id='call_9Rzbo72HIDOwZg6T5Ef0Z74d', name='search', type='function_call', id='fc_6871e89976648198a9403f65a26a6f090b8631d0fa3bbf21', status='completed'),
 ResponseFunctionToolCall(arguments='{"query":"tips for success in module 1"}', call_id='call_AG9lYW8rvWRfTm5hBIxzl7WH', name='search', type='function_call', id='fc_6871e899a3b881989f9fa0e24e027fc60b8631d0fa3bbf21', status='completed'),
 ResponseFunctionToolCall(arguments='{"query":"module 1 study strategies"}', call_id='call_iCMrBj7OI1J24VQeCxNRnCNN', name='search', type='function_call', id='fc_6871e89b15ac8198b668c2d72d0cd8170b8631d0fa3bbf21', status='completed')]

In [45]:

calls = response.output

In [46]:

for call in calls:
    result = do_call(call)
    chat_messages.append(call)
    chat_messages.append(result)

In [47]:
response = client.responses.create(
    model='gpt-4o-mini',
    input=chat_messages,
    tools=tools
)
response.output

[ResponseOutputMessage(id='msg_6871e914865081988f14ec6c18d593e10b8631d0fa3bbf21', content=[ResponseOutputText(annotations=[], text='To excel in Module 1, consider the following strategies:\n\n1. **Understand the Core Concepts**: Ensure that you have a solid grasp of Docker and Terraform basics as they are pivotal for the assignments.\n\n2. **Practice Environment Setup**: Follow the course instructions meticulously for setting up your development environment. If you encounter errors like "ModuleNotFoundError," troubleshoot them promptly by installing missing packages (e.g., `psycopg2`).\n\n3. **Utilize Resources**: Make use of available resources such as forums, FAQs, and additional documentation. For example, if you\'re using PostgreSQL, ensuring that you have the right modules (like `psycopg2` or `psycopg2-binary`) is crucial:\n   - You can install it using:\n     ```bash\n     pip install psycopg2-binary\n     ```\n\n4. **Work Through Examples**: Implement the provided examples and t

In [48]:

for entry in response.output:
    chat_messages.append(entry)
    print(entry.type)

    if entry.type == 'function_call':      
        result = do_call(entry)
        chat_messages.append(result)
    elif entry.type == 'message':
        print(entry.text) 

message


AttributeError: 'ResponseOutputMessage' object has no attribute 'text'

In [50]:
developer_prompt = """
You're a course teaching assistant. 
You're given a question from a course student and your task is to answer it.

Use FAQ if your own knowledge is not sufficient to answer the question.
When using FAQ, perform deep topic exploration: make one request to FAQ,
and then based on the results, make more requests.

At the end of each response, ask the user a follow up question based on your answer.
""".strip()

chat_messages = [
    {"role": "developer", "content": developer_prompt},
]

In [51]:
while True: # main Q&A loop
    question = input() # How do I do my best for module 1?
    if question == 'stop':
        break

    message = {"role": "user", "content": question}
    chat_messages.append(message)

    while True: # request-response loop - query API till get a message
        response = client.responses.create(
            model='gpt-4o-mini',
            input=chat_messages,
            tools=tools
        )

        has_messages = False
        
        for entry in response.output:
            chat_messages.append(entry)
        
            if entry.type == 'function_call':      
                print('function_call:', entry)
                print()
                result = do_call(entry)
                chat_messages.append(result)
            elif entry.type == 'message':
                print(entry.content[0].text)
                print()
                has_messages = True

        if has_messages:
            break

 How do i do well in module 1?


function_call: ResponseFunctionToolCall(arguments='{"query":"module 1"}', call_id='call_AwlupENIkkZREJmYxCXomoRu', name='search', type='function_call', id='fc_6871ea2beff4819b990161b37879f5a602972c9cb55c34b3', status='completed')

function_call: ResponseFunctionToolCall(arguments='{"query":"how to do well in module 1"}', call_id='call_hKW8jgU7rXkkudNwrzMy7wOE', name='search', type='function_call', id='fc_6871ea2ce5dc819b9abff27713be1cd002972c9cb55c34b3', status='completed')

function_call: ResponseFunctionToolCall(arguments='{"query":"module 1 tips"}', call_id='call_UZJz2CH9O3WmfoNdUvZkuduw', name='search', type='function_call', id='fc_6871ea2d2898819b8deb4ac666c1392102972c9cb55c34b3', status='completed')

To do well in Module 1 of your course, here are some key areas to focus on:

1. **Familiarize Yourself with Key Concepts**: Module 1 often covers foundational topics such as Docker and Terraform. Make sure you understand virtualization concepts, containerization, and infrastructure a

 stop
